In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

In [ ]:
df = pd.read_csv('/kaggle/input/national-stock-exchange-banking-sectors/NSE_BANKING_SECTOR.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
sns.set_theme(style="white")

corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))

heatmap = sns.heatmap(corr, cmap='BrBG', mask=mask, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':16}, pad=12)

In [ ]:
banks_list = df['SYMBOL'].unique().tolist()

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [ ]:
grouped_data = df.groupby('SYMBOL')
groups = grouped_data.groups

In [ ]:
day_name= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']

In [ ]:
def calculate_cagr(value_end, value_begin, time_in_years):
    try:
        return round((((value_end/value_begin)**(1/time_in_years)) - 1) * 100, 2)
    except:
        return 0
    return

In [ ]:
def get_cagr(grouped_data, bank_list):
    bank_directory = {}
    for bank in bank_list:
        close_prices = []
        dates = []
        df_bank = grouped_data.get_group(bank)
        df_bank.reset_index(drop=True, inplace=True)
        old_date = max(df_bank['DATE'])
        high_date = old_date
        low_date = min(df_bank['DATE'])
        dates.append(old_date)
        close_prices.append(df_bank.loc[df_bank['DATE'] == old_date].CLOSE.values[0])
        for _ in range(5):
            new_date = old_date - relativedelta(years=1)
            if new_date < low_date:
                break
            day = datetime.datetime.strptime(str(new_date), '%Y-%m-%d %H:%M:%S').weekday()
            if day_name[day] == 'Sunday':
                new_date = new_date + relativedelta(days=1)
            elif day_name[day] == 'Saturday':
                new_date = new_date - relativedelta(days=1)
                
            while new_date not in df_bank['DATE'].values:
                new_date = new_date - relativedelta(days=1)
                
            close_prices.append(df_bank.loc[df_bank['DATE'] == new_date].CLOSE.values[0])
            dates.append(new_date)
            old_date = new_date
            
        cagrs = [calculate_cagr(close_prices[i], close_prices[i+1], 1) for i in range(len(close_prices)-1)]
        
        gross_cagr = calculate_cagr(close_prices[0], close_prices[len(close_prices)-1], len(dates))
        
        bank_directory[bank] = {'min_date' : low_date, 'max_date' : high_date, 'gross_cagr' : gross_cagr,
                               'low_price' : min(close_prices) , 'high_price' : max(close_prices), 
                               'dates' : dates, 'cagrs' : cagrs, 'close_prices' : close_prices}
        
    return bank_directory
            
        

In [ ]:
bank_dir = get_cagr(grouped_data, banks_list)

In [ ]:
keys = list(bank_dir.keys())
gross_cagrs = [float(bank_dir[k]['gross_cagr']) for k in keys]
curr_price = [float(bank_dir[k]['close_prices'][0]) for k in keys]

In [ ]:
import plotly.express as px
fig = px.bar(x=keys, y=gross_cagrs, title="Gross Compunded Annual Growth Rate (Since 2016 or Inception)",
            labels={'x':'BANK NAME', 'y': 'CAGR %'}, hover_data={'Current CMP' : curr_price})

fig.show()